## Pięć zasad efektywnej inżynierii promptów
1. Jasne instrukcje poprawiają trafność.
2. Przykłady stabilizują odpowiedź.
3. Zdefiniowany format odpowiedzi = przewidywalny wynik.
4. Dzielenie na kroki = lepsze i pełniejsze rozwiązania.
5. Testy i weryfikacja = bezpieczeństwo i poprawność.

In [29]:
# Import bibliotek
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# Model bazowy
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Zasada 1 - Jasne instrukcje

In [30]:
# Zły prompt — niejasny, bez roli i oczekiwań
bad_prompt = "Napisz funkcję w Pythonie."
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content)

# Dobry prompt — jasno określona rola i oczekiwania
good_prompt = """Jesteś ekspertem programującym w Pythonie.
Napisz funkcję w Pythonie, która przyjmuje listę liczb całkowitych
i zwraca nową listę zawierającą tylko liczby parzyste.
Dodaj test jednostkowy w pytest."""
print("\n=== Dobry prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, żebym napisał? Możesz określić, co ma robić ta funkcja, jakie ma przyjmować argumenty i jakie ma zwracać wartości.

=== Dobry prompt ===
Oczywiście! Poniżej znajduje się funkcja w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodatkowo, dołączam test jednostkowy przy użyciu biblioteki `pytest`.

### Funkcja do filtrowania liczb parzystych

```python
def filtruj_parzyste(liczby):
    """Funkcja zwraca listę liczb parzystych z podanej listy."""
    return [liczba for liczba in liczby if liczba % 2 == 0]
```

### Test jednostkowy

Aby przetestować tę funkcję, możemy stworzyć plik testowy. Załóżmy, że nasza funkcja znajduje się w pliku `my_module.py`. Poniżej znajduje się kod testu:

```python
import pytest
from my_module import filtruj_parzyste

def test_filtruj_parzyste():
    assert filtruj_parzyste([1, 2, 3, 4, 5]) == [2, 4]
    assert filtruj_parzyste([10, 15, 20, 25]) == 

### Zasada 2 - Używaj przykładów

In [31]:
# Zadanie: wygeneruj tagi na podstawie treści strony firmy

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto propozycje tagów opisujących firmę Lego na podstawie podanego tekstu:

- Lego
- zabawki
- dzieci
- producent zabawek
- kreatywność
- edukacja
- zabawa
- konstrukcyjne
- rozwój dziecka
- marka
- innowacje
- zabawki edukacyjne
- rozrywka
- jakościowe zabawki
- budowanie

Jeśli potrzebujesz więcej tagów lub bardziej szczegółowych, daj znać!

=== One-shot ===
odzież, obuwie, sport

=== Few-shot ===
samochody, elektryczne, energia


In [32]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"U nas dostaniesz kawę na ławę!"
"Kawa szybko tanio i konkretnie!"
"""
print(llm.invoke(prompt).content)

Oto pięć sloganów reklamowych dla kawiarni w podobnym stylu:

1. "Kawa jak marzenie, smak w każdej kropli!"
2. "Zaparz swój poranek u nas – szybko i smacznie!"
3. "Kawa na każdą chmurę – humor w filiżance!"
4. "Odkryj smak radości – kawa serwowana z uśmiechem!"
5. "Z nami każdy łyk to przyjemność – kawa na wyciągnięcie ręki!"


In [33]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"Nasza kawa jest niczym spacer w rajskim ogrodzie w słoneczny radosny dzień."
"Pijąc naszą kawę poczujesz się tak, jakby anioł zatańczył Tobie na języku."
"""
print(llm.invoke(prompt).content)

1. "W każdej filiżance naszej kawy odkryjesz smak nieba, który przytula Twoje zmysły."
2. "Nasza kawa to magiczny eliksir, który ożywia duszę i rozświetla każdy dzień."
3. "Pijąc naszą kawę, poczujesz, jak serce tańczy w rytmie aromatycznych nut."
4. "W każdej kropli naszej kawy kryje się opowieść o przyjaźni, miłości i chwilach, które warto celebrować."
5. "Nasza kawa to słoneczny uśmiech, który rozgrzewa serce i sprawia, że każdy moment staje się wyjątkowy."


### Zasada 3 - Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [34]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Wrocław", days=2)

result: CityGuide = chain.invoke(req.model_dump())
print(result)
print(result.model_dump_json(indent=2))

city='Wrocław' summary='Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.' must_do=['Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.', 'Przejdź się po Ostrowie Tumskim i zwiedź Katedrę św. Jana Chrzciciela.', 'Zobacz Panoramę Racławicką, monumentalne dzieło sztuki.', 'Spędź czas w Hali Stulecia i ogrodach japońskich.', 'Zrób spacer po wrocławskich mostach i odkryj krasnale w mieście.']
{
  "city": "Wrocław",
  "summary": "Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.",
  "must_do": [
    "Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.",
    "Przejdź się po Ostrowie Tumskim i zwiedź Katedrę św. 

### Zasada 4 - Dziel złożone zadania na kroki

#### Przygotowanie planu wycieczki

In [35]:
# Zły prompt — wszystko naraz
bad_prompt = """Przygotuj trzydniowy plan zwiedzania Poznania z budżetem 300 euro,
uwzględniając atrakcje, restauracje, transport i mapy."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Dobry prompt — krok po kroku
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz mapy.

### Dzień 1: Stare Miasto i okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (około 5 euro)
- **Atrakcja:** Stary Rynek, Ratusz (bezpłatnie)
- **Transport:** Spacer po Starym Mieście (bezpłatnie)

**Południe:**
- **Atrakcja:** Muzeum Narodowe (bilet wstępu około 5 euro)
- **Obiad:** Restauracja "Bamberka" (około 10 euro)

**Popołudnie:**
- **Atrakcja:** Ostrów Tumski, Katedra (bezpłatnie, ewentualnie 2 euro za wejście do katedry)
- **Transport:** Sp ...

=== Dobry prompt — krok 1 ===
Oto lista najważniejszych atrakcji kulturalnych w Poznaniu wraz z ich godzinami otwarcia (godziny mogą się różnić w zależności od sezonu, dlatego zawsze warto sprawdzić aktualne informacje przed wizytą):

1. **Stary Rynek**
   - Godziny otwarcia: Całodobowo
   - Opis: Serce Poznania, z pięknymi kamienicami, ratuszem i fontanną Prozerpiny.

2. 

#### Generowanie artykułu w krokach

In [36]:
ARTICLE_SUBJECT = "Jakie kompetencje będą najcenniejsze u programistów za 10 lat."

plan = f"Przygotuj plan składający sie z 3 punktów artykułu na temat: {ARTICLE_SUBJECT}."
planGenerated = llm.invoke(plan).content
print(planGenerated, "...")

Oto plan artykułu na temat "Jakie kompetencje będą najcenniejsze u programistów za 10 lat":

1. **Umiejętności związane z sztuczną inteligencją i uczeniem maszynowym**  
   - Wprowadzenie do roli AI i ML w przyszłości programowania.  
   - Przykłady zastosowań AI w różnych branżach.  
   - Znaczenie zrozumienia algorytmów oraz umiejętności tworzenia i optymalizacji modeli.

2. **Zrozumienie architektury chmurowej i DevOps**  
   - Wyjaśnienie, jak chmura zmienia sposób, w jaki tworzymy i wdrażamy oprogramowanie.  
   - Kluczowe umiejętności związane z DevOps, takie jak automatyzacja, CI/CD i zarządzanie infrastrukturą.  
   - Przykłady narzędzi i technologii, które będą miały znaczenie w przyszłości.

3. **Umiejętności miękkie i współpraca w zespole**  
   - Znaczenie komunikacji i pracy zespołowej w kontekście złożonych projektów.  
   - Jak umiejętności interpersonalne wpływają na efektywność zespołów programistycznych.  
   - Rola liderów technicznych i mentorów w rozwijaniu talentó

In [37]:
MAX_WORDS_IN_PARAGRAPH = 200

prevParagraphContent = ""

for i in range(1,4):
    paragraphPrompt = f"Przygotuj tylko treść {i}. akapitu artykułu: {ARTICLE_SUBJECT}. Maksymalna długość tekstu to {MAX_WORDS_IN_PARAGRAPH}. Wygeneruj tylko tekst jednego paragrafu. Plan całego artykułu:\n {planGenerated}. Treść poprzednich akapitów: {prevParagraphContent}."
    paragraphContent = llm.invoke(paragraphPrompt).content
    prevParagraphContent += paragraphContent + "\n"
    print(f"Akapit {i}.", paragraphContent, "\n\n")

Akapit 1. W ciągu najbliższych dziesięciu lat kompetencje programistów będą musiały ewoluować, aby sprostać rosnącym wymaganiom rynku technologicznego. W szczególności umiejętności związane z sztuczną inteligencją (AI) i uczeniem maszynowym (ML) staną się kluczowe, ponieważ coraz więcej branż wdraża te technologie w celu automatyzacji procesów i analizy danych. Programiści, którzy potrafią nie tylko korzystać z gotowych rozwiązań, ale także rozumieć algorytmy oraz tworzyć i optymalizować modele AI, będą w czołówce poszukiwanych specjalistów. W miarę jak AI i ML będą integrowane w różnorodne aplikacje, umiejętność ich implementacji stanie się niezbędna dla każdego, kto pragnie odnaleźć się w dynamicznie zmieniającym się świecie technologii. 


Akapit 2. Zrozumienie architektury chmurowej oraz praktyk DevOps będzie kolejnym kluczowym obszarem, w którym programiści muszą się rozwijać. Chmura zmienia sposób, w jaki tworzymy, wdrażamy i zarządzamy oprogramowaniem, umożliwiając elastyczność 

### Zasada 5 - Testuj i weryfikuj wyniki

In [38]:
!pip install -q langchain langchain-openai langchain-community python-dotenv

In [39]:
from langchain_classic.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="coś zupełnie innego"
)

print(result)


{'score': 0.2288585785726367}
